**Статистическая языковая модель**

Евгений Борисов <esborisov@sevsu.ru>

подбираем наиболее вероятное продолжение цепочки слов

In [2]:
import gzip
import requests
from bs4 import BeautifulSoup

In [3]:
url='http://lib.ru/NEWPROZA/LOBAS/taxisty.txt'
text = BeautifulSoup(requests.get(url).text).get_text()
with gzip.open('taxisty.txt.gz','wt') as f: f.write(text)

# with gzip.open('taxisty.txt.gz','rt') as f: text = f.read()

text = text[1030:-7261].strip() # выкидываем заголовок и хвост страницы 
print(f'символов:{len(text)}\n---------------\n'%())
print(text[:343])

символов:637765
---------------

Вы прилетели в  Нью-Йорк и  остановились  в  одном  из отелей, глядящих
окнами на Центральный парк.  Наутро по  приезде вы вышли из  отеля, вдохнули
полной грудью очищенный зеленью парка воздух  и,  взглянув на часы, --  пора
было начинать хлопотливый день, --  направились к  первому из  таксомоторов,
выстроившихся вереницей у подъезда.
    


In [4]:
# url='http://az.lib.ru/d/dostoewskij_f_m/text_0080.shtml'
# text = BeautifulSoup(requests.get(url).text).get_text()
# with gzip.open('dostoewskij.txt.gz','wt') as f: f.write(text)

# # with gzip.open('dostoewskij.txt.gz','rt') as f: text = f.read()

# text = text[2876:-664184].strip() # выкидываем заголовок и хвост страницы 
# print(f'символов:{len(text)}\n---------------\n'%())
# print(text[:355])

In [5]:
from nltk import __version__ as nltk_version
print('nltk version:',nltk_version)

nltk version: 3.8.1


In [6]:
from tqdm.auto import tqdm
from random import sample

from nltk.tokenize import sent_tokenize as nltk_sentence_split
from nltk.tokenize import word_tokenize as nltk_tokenize_word

sentences = [ 
    nltk_tokenize_word(s) # разбиваем предложения на слова
    for s in tqdm(nltk_sentence_split(text)) # режем текст на отдельные предложения
]

print('предложений: %i\n'%(len(sentences)))
display( sample(sentences,1) )

  0%|          | 0/6685 [00:00<?, ?it/s]

предложений: 6685



[['Он', 'не', 'упомянул', 'Англию', '.']]

In [7]:
sentences = sentences[:128] # ограничиваем датасет для ускорения процеса 

---

In [8]:
from itertools import chain
from collections import Counter

text_tokens = list(chain(*sentences)) # собираем все токены (слова) из текста
vocab_size = len(set(text_tokens)) # размер словаря

text_tokens_freq = Counter( text_tokens ) # оценка частоты использования слов

print('всего слов тексте: %i'%(len(text_tokens)))
print('размер словаря: %i'%(vocab_size))
text_tokens_freq.most_common()[:10] # наиболее частые токены

всего слов тексте: 3685
размер словаря: 1651


[(',', 449),
 ('и', 111),
 ('.', 88),
 ('в', 74),
 ('--', 62),
 ('не', 62),
 ('на', 39),
 ('с', 38),
 (':', 34),
 ('что', 33)]

----

In [9]:
# from nltk.util import bigrams
from nltk.util import ngrams as nltk_ngrams

# вынимаем все n-gram из текста
ngram_len = 2 # работаем с биграммами
text_ngrams = [ ngram for s in sentences for ngram in nltk_ngrams(s,ngram_len) ]
print('количество n-gram: %i'%(len(set(text_ngrams))))
sample(text_ngrams,5)

количество n-gram: 3135


[('скорее', 'забыть'),
 ('рука', 'моя'),
 ('почем', 'и'),
 ('к', 'этому'),
 ('Разве', 'вы')]

In [14]:
# cчитаем частоту n-gram
text_ngrams_freq = Counter(text_ngrams)
sample( list(text_ngrams_freq.items()), 7)

[(('побойчей', ','), 1),
 (('до', 'лица'), 1),
 (('двадцать', 'минут'), 1),
 (('услужливо', 'распахнул'), 1),
 (('Лимузин', '--'), 1),
 (('случиться', 'и'), 1),
 (('...', 'Не'), 1)]

In [15]:
text_ngrams_freq.most_common()[:10] # наиболее частые n-ngram

[((',', 'и'), 33),
 ((',', 'а'), 24),
 ((',', 'что'), 23),
 ((',', 'как'), 22),
 (('и', ','), 10),
 ((',', 'когда'), 10),
 (("''", ','), 10),
 ((',', 'я'), 9),
 ((',', '--'), 8),
 ((',', 'но'), 8)]

----

Оценка вероятностей совместного использования слов со сглаживанием Лапласа


$$
P(w_n|w_{n-1}) = \frac{ C(w_{n-1} w_{n}) +1 }{ C(w_{n-1}) + V }
$$

In [16]:
text_ngrams_prob = { # оценка совместного использования токенов
    ngram : (text_ngrams_freq[ngram]+1) / ( text_tokens_freq[ ngram[0] ] + vocab_size )
    for ngram in text_ngrams_freq 
}

display( text_ngrams_prob )

{('Вы', 'прилетели'): 0.0012091898428053204,
 ('прилетели', 'в'): 0.0012106537530266344,
 ('в', 'Нью-Йорк'): 0.0011594202898550724,
 ('Нью-Йорк', 'и'): 0.0012084592145015106,
 ('и', 'остановились'): 0.0011350737797956867,
 ('остановились', 'в'): 0.0012106537530266344,
 ('в', 'одном'): 0.0017391304347826088,
 ('одном', 'из'): 0.0018148820326678765,
 ('из', 'отелей'): 0.001195457262402869,
 ('отелей', ','): 0.0012099213551119178,
 (',', 'глядящих'): 0.0009523809523809524,
 ('глядящих', 'окнами'): 0.0012106537530266344,
 ('окнами', 'на'): 0.0012106537530266344,
 ('на', 'Центральный'): 0.0017751479289940828,
 ('Центральный', 'парк'): 0.0018148820326678765,
 ('парк', '.'): 0.0018148820326678765,
 ('Наутро', 'по'): 0.0012106537530266344,
 ('по', 'приезде'): 0.001201923076923077,
 ('приезде', 'вы'): 0.0012106537530266344,
 ('вы', 'вышли'): 0.0011997600479904018,
 ('вышли', 'из'): 0.0018148820326678765,
 ('из', 'отеля'): 0.0017931858936043037,
 ('отеля', ','): 0.0030175015087507543,
 (',', 'вд

In [17]:
# оценка вероятности новых ngram по статистике text_ngrams_prob
def get_ngram_prob(ngram,text_ngrams_prob=text_ngrams_prob,text_tokens_freq=text_tokens_freq):
    if ngram in text_ngrams_prob: return text_ngrams_prob[ngram]
    token_0_freq = text_tokens_freq[ngram[0]] if ngram[0] in text_tokens_freq else 0.
    return 1./(token_0_freq+len(text_tokens_freq))

выбираем дополнение предложения с наибольшей вероятностью цепочки слов

$$
P(w_1,\ldots, w_n) = \prod_{k=1}^{n} P(w_k|w_k-1)
$$

In [18]:
from operator import mul
from functools import reduce 

# оценка предложения
def get_sentence_prob(
        sentence,
        text_ngrams_prob=text_ngrams_prob,
        text_tokens_freq=text_tokens_freq,
        ngram_len=ngram_len
    ):
    return reduce( mul, [ 
        get_ngram_prob(ngram,text_ngrams_prob=text_ngrams_prob,text_tokens_freq=text_tokens_freq)
        for ngram in nltk_ngrams(sentence,ngram_len) 
    ] )


In [19]:
# оценка всех возможных продолжений предложения
def get_next_token_prob(sentence,text_ngrams_prob=text_ngrams_prob,text_tokens_freq=text_tokens_freq):
    
    sentence_prob = get_sentence_prob( # оценка предложения
            sentence,
            text_ngrams_prob=text_ngrams_prob,
            text_tokens_freq=text_tokens_freq
        )
    
    token_next = { # оценки всех возможных продолжений
        ngram[1] : 
            sentence_prob*
              get_ngram_prob(ngram,text_ngrams_prob=text_ngrams_prob,text_tokens_freq=text_tokens_freq)
        for ngram in text_ngrams_prob if ngram[0]==sentence[-1] 
    }
    
    return token_next

In [20]:
from collections import OrderedDict

def get_top_prob_token(tokens_prob,n=3):  # n наиболее вероятных продолжений
    kf = lambda k: tokens_prob[k]
    return OrderedDict([
        (key,tokens_prob[key])
        for key in sorted(tokens_prob, key=kf, reverse=True)[:n]
    ])

----

In [21]:
# генерируем продолжения

for sentence in sample(sentences,30):
    if len(sentence)<10: continue
     # берём начало предложения
    sentence_ = sentence[:-(len(sentence)//4)]
    
    # генерируем возможные продолжения
    
    # считаем верояности продолжений
    next_token_prob = get_next_token_prob(sentence_) 
    
    # выбираем наиболее вероятные
    top_prob_token = get_top_prob_token(next_token_prob)
       
    print( 
        ' '.join(sentence_)
        +  ' ... ' 
        + '{ '
        + ' | '.join(top_prob_token.keys())
        + ' }'
        + '\n'
    )

Не калеча ее ни каторжными трудами , ни многолетним скопидомством , а с помощью одного безупречно легального способа , который сработает даже при условии , что у тебя нет сбережений ... И если тебя интересует только это , тебе не придется ни читать мою книжку , ни даже листать ее , выискивая нужный совет : открывай сразу главу 20 , и через двадцать минут ты будешь знать все необходимое и лишь слегка подивишься тому , до чего это просто , да , может , в уголке твоего сознания мелькнет мысль о том , как много твоих знакомых , которых ты издавна и искренне привык уважать , проделали в свое время ... { `` | бурчит | , }

Отвернется в закуток , проглотит ломоть жирного одесского штруделя с изюмом , закусит хвостом маринованной селедки и , ... { и | а | что }

Сомнений не было : Ангел давал аэропорты , но , оказывается , денег он ... { не | вызван | может }

Раз уж так вышло , что речь почему-то зашла о швейцарах , давай я познакомлю тебя , читатель , со ... { всеми | мной }

Вот и скажите т